In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates
%matplotlib inline
import statsmodels.api as sm
import seaborn as sns
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go

import cufflinks as cf # Connects Plotly to Pandas
# Makes Plotly work in your Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [54]:
from statsmodels.tsa.stattools import adfuller

def adf_test(TimeSerie_Data):
    test = adfuller(TimeSerie_Data,autolag='AIC')
    print('Augmented Dickey-Fuller:')
    test_result = pd.Series(test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    return test_result

# Vector autoregressive models
Vector autoregressive models are used when you want to predict multiple time series data and using in one model.<br>
This differs from ARIMA in that we will have to check for stationarity on our own. We will make the data stationary and then invert it back to its original form.<br>
We'll use AIC to measure our model. As the model increases in complexity it will get more accurate up to a point where AIC will punish that complexity.

In [65]:
from statsmodels.tsa.api import VAR

gdp = pd.read_csv('./dataset/gdp-data.csv',index_col='date',parse_dates=True)
gdp = gdp[gdp.index<'2020-03-31']
gdp.head()

,gdp,biz-apps,prod
date,,,
2004-09-30,12522.425,574777,89.09
2004-12-31,12761.337,583203,89.82
2005-03-31,12910.022,610664,89.53
2005-06-30,13142.873,618094,89.85
2005-09-30,13332.316,630382,90.22


In [66]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=gdp.index,y=gdp.gdp,mode='lines',name='GDP'))
# biz-apps表新成立的business-app
fig.add_trace(go.Scatter(x=gdp.index,y=gdp['biz-apps'] * 0.025,mode='lines',name='New Business-App')) 
fig.add_trace(go.Scatter(x=gdp.index,y=gdp['prod'] * 180,mode='lines',name='Productivity'))
fig.update_layout(title='Economic Measurement',
xaxis_title='Date',yaxis_title='Amount')

# 可以看出資料並非Stationary需要做轉換

# Difference the data

In [67]:
gdp_diff = gdp.diff()
gdp_diff.head()

,gdp,biz-apps,prod
date,,,
2004-09-30,NaN,NaN,NaN
2004-12-31,238.912,8426.0,0.73
2005-03-31,148.685,27461.0,-0.29
2005-06-30,232.851,7430.0,0.32
2005-09-30,189.443,12288.0,0.37


In [68]:
gdp_diff = gdp_diff.dropna()

fig = go.Figure()
fig.add_trace(go.Scatter(x=gdp_diff.index,y=gdp_diff.gdp,mode='lines',name='GDP'))
# biz-apps表新成立的business-app
fig.add_trace(go.Scatter(x=gdp_diff.index,y=gdp_diff['biz-apps'] * 0.025,mode='lines',name='New Business-App')) 
fig.add_trace(go.Scatter(x=gdp_diff.index,y=gdp_diff['prod'] * 180,mode='lines',name='Productivity'))
fig.update_layout(title='Difference of Economic Measurement',xaxis_title='Date',yaxis_title='Difference')

In [69]:
# adf test
print(adf_test(gdp_diff['gdp']))
print(adf_test(gdp_diff['biz-apps']))
print(adf_test(gdp_diff['prod']))

Augmented Dickey-Fuller:
Test Statistic                 -4.231840
p-value                         0.000582
#Lags Used                      0.000000
Number of Observations Used    60.000000
dtype: float64
Augmented Dickey-Fuller:
Test Statistic                 -4.847785
p-value                         0.000044
#Lags Used                      1.000000
Number of Observations Used    59.000000
dtype: float64
Augmented Dickey-Fuller:
Test Statistic                -8.342130e+00
p-value                        3.155371e-13
#Lags Used                     0.000000e+00
Number of Observations Used    6.000000e+01
dtype: float64


# Train and Test

In [88]:
train_gdp = gdp_diff[:48]
test_gdp = gdp_diff[48:]

# auto_arima doesn't work for vector autoregressions so we must find p by checking
# for the lowest value of AIC
model = VAR(train_gdp)

# lag set to 7
for lag in range(8):
    result = model.fit(lag)
    print(f'Order : {lag}, AIC : {result.aic}')
    
# lag = 1 is ok
model_fit = model.fit(1)
model_fit.summary()

Order : 0, AIC : 27.374924449199494
Order : 1, AIC : 26.293083666632572
Order : 2, AIC : 26.494887400642295
Order : 3, AIC : 26.532437084592964
Order : 4, AIC : 26.542914646077723
Order : 5, AIC : 26.254539569966997
Order : 6, AIC : 26.218293993352248
Order : 7, AIC : 26.408782468222242


C:\Users\Mashiro\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:527: ValueWarning:

No frequency information was provided, so inferred frequency Q-DEC will be used.



  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 02, Sep, 2021
Time:                     03:06:38
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    26.7655
Nobs:                     47.0000    HQIC:                   26.4708
Log likelihood:          -805.958    FPE:                2.62711e+11
AIC:                      26.2931    Det(Omega_mle):     2.05618e+11
--------------------------------------------------------------------
Results for equation gdp
                 coefficient       std. error           t-stat            prob
------------------------------------------------------------------------------
const              64.425240        22.976489            2.804           0.005
L1.gdp              0.658488         0.163375            4.031           0.000
L1.biz-apps        -0.000813         0.000817           -0.995        